In [ ]:
%pylab notebook

In [ ]:
import spt3g
from spt3g import core, dfmux

In [ ]:
for frame in core.G3File('/spt/data/bolodata/downsampled/calibrator/3009897/0000.g3'):
    print(frame)
    #print(frame.type)
    #print(frame.keys())

In [ ]:
frames = list(core.G3File('/spt/data/bolodata/downsampled/calibrator/3009897/0000.g3'))

In [ ]:
len(frames)

In [ ]:
scan = frames[-1]

In [ ]:
scan.keys()

In [ ]:
ts = frames[-1]['RawTimestreams_I']['W136/2017.W136.5.4.7.912']

In [ ]:
ts.sample_rate / core.G3Units.Hz

In [ ]:
cal = scan['CalibratorOn']

In [ ]:
scan['CalibratorOn'].sample_rate / core.G3Units.Hz

In [ ]:
figure()
hz = scan['CalibratorOn'].sample_rate / core.G3Units.Hz
times = np.arange(len(scan['CalibratorOn'])) / hz
plot(times, scan['CalibratorOn'], '.')
ylim(-0.1,1.1)

In [ ]:
figure()
pad = 10
plot(times[pad:-pad], scan['RawTimestreams_I']['W136/2017.W136.5.4.7.912'][pad:-pad], '.')

In [ ]:
figure()
psd(ts[pad:-pad], Fs=hz, NFFT=2**10, window=window_hanning, detrend='linear');

In [ ]:
figure()
psd(cal[pad:-pad], Fs=hz, NFFT=2**10, window=window_hanning, detrend='linear');

In [ ]:
scan['DfMuxHousekeeping'].keys()

In [ ]:
boardinfo = scan['DfMuxHousekeeping'][156]

In [ ]:
list(boardinfo.temperatures.iteritems())

In [ ]:
dfmux.HousekeepingForBolo?

In [ ]:
hkmap = scan['DfMuxHousekeeping']
wiringmap = frames[1]["WiringMap"]
bolo = 'W136/2017.W136.5.4.7.912'
(board, mezz, module, channel) = dfmux.HousekeepingForBolo(hkmap, wiringmap, bolo, all_hk=True)

In [ ]:
(channel.carrier_amplitude, channel.state)

In [ ]:
def get_calibrator_freq(caltimestream):
    # Convert to and from Hz for easy printing/plotting/debugging
    hz = caltimestream.sample_rate / core.G3Units.Hz
    freqs = np.fft.rfftfreq(len(caltimestream)) * hz
    rfft = np.fft.rfft(caltimestream-np.mean(caltimestream))
    freq = freqs[np.argmax(rfft)] # in Hz
    return freq * core.G3Units.Hz

def get_response_at_freq(bolotimestream, freq, pad=10):
    bolotimestream = bolotimestream[pad:-pad] # lop off the first and last few samples
    freqs = np.fft.rfftfreq(len(bolotimestream)) * bolotimestream.sample_rate
    freqindex = np.argmin( np.abs(freqs-freq) )
    modts = matplotlib.mlab.detrend_linear(bolotimestream) * np.hanning(len(bolotimestream))
    rfft = np.fft.rfft(modts)
    response = np.abs(rfft[freqindex])
    return response

class TunedCalResponse(object):
    def __init__(self, output='CalibratorResponse'):
        self.wiringmap = None
        self.output = output
        self.responses = None
    def __call__(self, frame):
        #print(frame)
        if frame.type == core.G3FrameType.Wiring:
            self.wiringmap = frame['WiringMap']
        if frame.type == core.G3FrameType.Scan:
            hkmap = frame['DfMuxHousekeeping']
            calibrator_freq = get_calibrator_freq(frame['CalibratorOn'])
            print('calibrator_freq:', calibrator_freq / core.G3Units.Hz)
            frame['CalibratorResponse'] = core.G3MapDouble()
            for bolo in frame['RawTimestreams_I'].keys():
                hkmap = frame['DfMuxHousekeeping']
                status = dfmux.HousekeepingForBolo(hkmap, self.wiringmap, bolo)
                #if status.state == 'tuned':
                if status.carrier_amplitude != 0.0:
                    resp = get_response_at_freq(frame['RawTimestreams_I'][bolo], calibrator_freq)
                    #print(bolo, resp)
                    frame['CalibratorResponse'][bolo] = resp
            self.responses = frame['CalibratorResponse']

In [ ]:
pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename='/spt/data/bolodata/downsampled/calibrator/3009897/0000.g3')
tunedcalresponse = TunedCalResponse()
pipe.Add(tunedcalresponse)
pipe.Run()

In [ ]:
responses = np.asarray(tunedcalresponse.responses.values())

In [ ]:
figure()
minresp = np.percentile(responses, 1)
maxresp = np.percentile(responses, 99)
hist(np.log10(responses[(responses>=minresp)*(responses<=maxresp)]), bins=100);